In [1]:
# This is an internal notebook that generates the quiz notebooks

In [2]:
import toml
import nbformat as nbf

def _add_quiz_cell(notebook, sourcefile, language="ja"):
    with open(sourcefile) as f:
        question = toml.load(sourcefile)
    
    # Question cell
    text = (
        '<h3>{title}</h3>'
        '{tags}\n'
        '<br>\n{content}'
        '{hint}\n'
        '<details><summary>Answer</summary>'
        '<pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;">'
        '<code style="background-color: #25292f; color: #ffffff">{answer}</code>'
        '</pre></details>'
    )    
    title = question["title"][language]
    content = question["content"][language]
    tags = " ".join(
      f'<span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">{t}</span>'
      for t in question["tags"])
    hint = question["hint"][language].strip()
    if hint != "":
        hint = f"<details><summary>Hint</summary>{hint}</details>"
    answer = question["judgement"]["query"].strip()   
    text = text.format(title=title, content=content, tags=tags, hint=hint, answer=answer)
    notebook.cells.append(nbf.v4.new_markdown_cell(text))
    
    # Code cell
    code = (
        "%%tdquiz\n"
        "/* {instruction} */\n"
        "\n\n\n\n"
    )
    instruction = "クエリを下記に記載" if language == "ja" else "Write your query blow"
    code = code.format(instruction=instruction)
    notebook.cells.append(nbf.v4.new_code_cell(code))


def generate_notebooks(sourcedir, savedir, language="ja"):
    import os
    import csv
    import nbformat as nbf

    # Load categories
    with open(os.path.join(sourcedir, "_categories.csv")) as f:
        reader = csv.reader(f)
        header = next(reader)
        categories = [dict(zip(header, row)) for row in reader]

    # Make a notebook for each category
    notebooks = {}
    for c in categories:
        id = c["category_id"]
        if language == "ja":
            filename = "{}_{}.ipynb".format(id.zfill(2), c["category_en"])
        else:
            filename = "{}-{}_{}.ipynb".format(language, id.zfill(2), c["category_en"])
        title = c["category_ja"] if language == "ja" else c["category_en"]
        nb = nbf.v4.new_notebook()
        nb.cells.append(nbf.v4.new_markdown_cell(f"# {title}"))
        
        if language == "ja":
            code = (
                "# このセルはそのまま実行して、tdquiz コマンドをロードしてください。\n"
                "# tdquiz コマンドはクエリを実行してその結果を返します。\n"
                "%load_ext tdquiz\n\n"
                "# tdquiz コマンドをテスト\n"
                "%tdquiz SELECT USER, SESSION, CURRENT_TIMESTAMP"
            )
        else:
            code = (
                "# Run this cell to load the tdquiz magic command, that runs queries and return the results.\n"
                "%load_ext tdquiz\n\n"
                "# Test tdquiz magic command\n"
                "%tdquiz SELECT USER, SESSION, CURRENT_TIMESTAMP"
            )
        nb.cells.append(nbf.v4.new_code_cell(code))
        notebooks[id] = {"id": id, "filename": filename, "notebook": nb}

    # Load questions
    with open(os.path.join(sourcedir, "_questions.csv")) as f:
        reader = csv.reader(f)
        header = next(reader)
        questions = [dict(zip(header, row)) for row in reader]

    for q in questions:
        category_id = q["category_id"]
        notebook = notebooks[category_id]["notebook"]
        sourcefile = os.path.join(sourcedir, q["filename"] + ".toml")
        _add_quiz_cell(notebook, sourcefile, language=language)

    # Save notebooks
    for n in notebooks.values():
        savepath = os.path.join(savedir, n["filename"])
        print(savepath)
        os.makedirs(os.path.abspath(os.path.dirname(savepath)), exist_ok=True)
        nbf.write(n["notebook"], savepath)


sourcedir = "../questions"
savedir = "../"
generate_notebooks(sourcedir, "../", "ja")
generate_notebooks(sourcedir, "../", "en")

../01_SQL Basic.ipynb
../02_SQL Intermediate.ipynb
../03_Database Information.ipynb
../04_Geospatial Analysis.ipynb
../05_Statistics.ipynb
../06_Math and Puzzle.ipynb
../en-01_SQL Basic.ipynb
../en-02_SQL Intermediate.ipynb
../en-03_Database Information.ipynb
../en-04_Geospatial Analysis.ipynb
../en-05_Statistics.ipynb
../en-06_Math and Puzzle.ipynb
